In [123]:
%run 'init.ipynb'
%run 'functions.ipynb'
%run 'geometry_checks.ipynb'

225.53


In [149]:
N = 1000000
mass_fv = 40000

materials = [
    "SS_InnerCryostat",
    "SS_OuterCryostat",
    "OuterCryostatReflector",
    "SS_BellPlate",
    "SS_BellSideWall",
    "PmtTpc",
    "Copper_TopRing",
    "Copper_LowerRing",
    "Teflon_Pillar_",
    "Copper_FieldGuard_",
    "Copper_FieldShaperRing_",
    "Electrodes",
    "Teflon_BottomTPC",
    "Teflon_TPC",   
    "Top_PTFE_Frame",
    "Copper_BottomPmtPlate",
    "Copper_TopPmtPlate",
    ]



DATE = 'XENONnT_2019-11-04'

isotope_array = ["U238Pb206",
                "K40",
                "Th228Pb208",
                "U235Pb207",
                "Co60",
                "Cs137",]
num_file = '1'

array = np.arange(1, 2)



In [125]:
rootfile = '/userdata/arocchetti/' + DATE + '/'+ material + '/' + isotope + '/' + 'output_'+ material + '_' + isotope +  '_' + num_file+'_Sort' + '.root' 
def get_typepri(rootfile):
    f = ROOT.TFile(rootfile)
    tree = f.Get('events/events')
    out=root_numpy.tree2array(tree)
    typepri = np.array([out['typepri'][i] for i in range(len(out['typepri']))])
    return typepri
    f.Close()

In [126]:
#rewrite the function for nSort files 
def get_xyz(rootfile):
    file = uproot.open(rootfile)
    tree = file["events/events"]
    df = tree.pandas.df([ "xpri", "ypri", "zpri", "epri", "ns", "X", "Y", "Z", "Ed", ])
    df.columns = ['xp', 'yp', 'zp_uc', "epri", "ns", "X", "Y", "Z_uc", "Ed"] #rename 
    #Add some variables
    df['rp'] = np.sqrt(df.xp*df.xp+ df.yp*df.yp)
    df['r2p'] = df.rp*df.rp
    df['R'] = np.sqrt(df.X*df.X+ df.Y*df.Y)
    df['R2'] = (df.R*df.R)
    offset = 1488/2
    df['Z'] = df.Z_uc+offset
    df['zp'] = df.zp_uc+ offset
    typepri = get_typepri(rootfile)
    df['typepri'] = typepri
    return df

In [127]:
material == 'SS_OuterCryostat'
rootfile = '/userdata/arocchetti/' + DATE + '/'+ material + '/' + isotope + '/' + 'output_'+ material + '_' + isotope +  '_' + num_file+'_Sort' + '.root'
data = get_xyz(rootfile)





In [128]:
def get_electrodes():    
    electrodes = [
            "SS_GateRing",
            "SS_AnodeRing",
            "SS_TopMeshRing",
            "SS_CathodeRing",
            "SS_BottomMeshRing",
            ]

    for electrode in electrodes:

        if  (electrode == "SS_GateRing"):
            rootfile = '/userdata/arocchetti/' + DATE + '/'+ electrode + '/' + isotope + '/' + 'output_'+ electrode + '_' + isotope +  '_' + num_file+'_Sort' + '.root'
            data = get_xyz(rootfile)

        if  (electrode =="SS_AnodeRing") :
            rootfile = '/userdata/arocchetti/' + DATE + '/'+ electrode + '/' + isotope + '/' + 'output_'+ electrode + '_' + isotope +  '_' + num_file+'_Sort' + '.root'
            data_anode = get_xyz(rootfile)
            data = pd.concat([data, data_anode])

        if (electrode == "SS_TopMeshRing"):
            rootfile = '/userdata/arocchetti/' + DATE + '/'+ electrode + '/' + isotope + '/' + 'output_'+ electrode + '_' + isotope +  '_' + num_file+'_Sort' + '.root'
            data_topmesh = get_xyz(rootfile)
            data = pd.concat([data, data_topmesh]) 

        if (electrode == "SS_CathodeRing"):
            rootfile = '/userdata/arocchetti/' + DATE + '/'+ electrode + '/' + isotope + '/' + 'output_'+ electrode + '_' + isotope +  '_' + num_file+'_Sort' + '.root'
            data_cathode = get_xyz(rootfile)
            data = pd.concat([data, data_cathode])   

        if (electrode == "SS_BottomMeshRing"):
            rootfile = '/userdata/arocchetti/' + DATE + '/'+ electrode + '/' + isotope + '/' + 'output_'+ electrode + '_' + isotope +  '_' + num_file+'_Sort' + '.root'
            data_bottommesh = get_xyz(rootfile)
            data = pd.concat([data, data_bottommesh]) 
            return data
        
def get_TopPFTE():
    parts = [
        "GXeTeflon_TopElectrodesFrame",
        "Teflon_TopElectrodesFrame"]
    for part in parts:
        if (part == "GXeTeflon_TopElectrodesFrame"):
            rootfile = '/userdata/arocchetti/' + DATE + '/'+ part + '/' + isotope + '/' + 'output_'+ part + '_' + isotope +  '_' + num_file+'_Sort' + '.root'
            data = get_xyz(rootfile)   
        if (part == "Teflon_TopElectrodesFrame"):
            rootfile = '/userdata/arocchetti/' + DATE + '/'+ part + '/' + isotope + '/' + 'output_'+ part + '_' + isotope +  '_' + num_file+'_Sort' + '.root'
            data2 = get_xyz(rootfile)
            data = pd.concat([data, data2]) 
            return data        


In [129]:
# divide the Cryostat
material == "SS_OuterCryostat"
def divide_outercryo(data):
    flange1 = data[(data.R>815) & (data.zp > 300+offset)]
    flange2 = data[(data.R>815) & (data.zp < 300+offset) & (data.zp > -520 + offset)] 
    flange3 = data[(data.R>815) & (data.zp < -520+offset) & (data.zp > -1000 + offset)]
    shell = data[(data.R<815) & (data.zp > -950)]
    elongation = data[(data.R<815) & (data.zp < -950)]
    return flange1, flange2, flange3, shell, elongation
    

flange1, flange2, flange3, shell, elongation = divide_outercryo(data)
flanges = pd.concat([flange1, flange2, flange3])

In [130]:
def calculate_events(data):
    ss = single_scatter(data)
    fv_ = fv(ss)
    en = energy(fv_, 1, 12)
    events_passing = len(en)
    return events_passing

n = calculate_events(shell)   
print(n)

# "OuterCryostat_flanges":  OuterCryostatShell     parameters = get_parameters(material)

9


In [131]:
material ="OuterCryostat_Elongation"
def calculate_rate(data, material, n_gen, m_fv):
    n_events = calculate_events(data)
    E = 12
    p = get_parameters(material)
    mass = p['mass']
    print("material= ", material)
    print("mass =", p['mass'], "gr")
    print("---------------------")
    for isotope in isotope_array:
        print(isotope)
        contamination = p[isotope]
        if (contamination == 0):
            print("No contamination value for:", isotope)
        else:
            time = n_gen / (mass * contamination)
            rate = n_events/(time * m_fv * E)
            alpha = mass / (m_fv * E * n_gen)
            err = "err_"+isotope
            err = str(err)
            print("contamination for", isotope, ":", p[isotope], "+-",p[err] )
            error_rate = alpha * np.sqrt((n_events * p[err])**2 + (contamination*np.sqrt(n_events))**2)
            print("---------> Rate ev:", rate, "+-", error_rate)


In [157]:
materials = [
    "Teflon_Pillar_", #not work ----> nSort Failed, redo the job
    "Copper_FieldGuard_",#modify the _
    "Teflon_TPC",  #new  
    "Top_PTFE_Frame",#new
    "Copper_BottomPmtPlate",#new
    "Copper_TopPmtPlate"#new,
    ]
materials = [
    'Teflon_Pillar_']
for material in materials:
    rootfile = '/userdata/arocchetti/' + DATE + '/'+ material + '/' + isotope + '/' + 'output_'+ material + '_' + isotope +  '_' + num_file+'_Sort' + '.root'
    print(rootfile)
    #data = get_xyz(rootfile)
    #calculate_rate(data, material, N, mass_fv)


/userdata/arocchetti/XENONnT_2019-11-04/Teflon_Pillar_/Co60/output_Teflon_Pillar__Co60_1_Sort.root


In [154]:
materials = [
    'Teflon_Pillar_']
for material in materials:
    
    if material == "Top_PTFE_Frame":
        data = get_TopPFTE()
        material = "Top_PTFE_Frame"
        calculate_rate(data, material, N, 4000)
    
    elif material == "Electrodes":
        data = get_electrodes()
        material = "SS_Electrodes"
        calculate_rate(data, material, N, 4000)
    
    elif material == 'SS_OuterCryostat':
        rootfile = '/userdata/arocchetti/' + DATE + '/'+ material + '/' + isotope + '/' + 'output_'+ material + '_' + isotope +  '_' + num_file+'_Sort' + '.root'
        data = get_xyz(rootfile)
        flange1, flange2, flange3, shell, elongation = divide_outercryo(data)
        ####################################
        flange = pd.DataFrame()
        flanges = pd.concat([flange1, flange2, flange3])
        #calculate_rate(flanges, "OuterCryostat_flanges", N, mass_fv)
        #print("###########################################")
        #calculate_rate(shell, "OuterCryostatShell", N, mass_fv)
        #print("###########################################")
        #calculate_rate(elongation, "OuterCryostat_Elongation", N, mass_fv)
        #print("###########################################")
    
    elif material == 'SS_InnerCryostat':
        rootfile = '/userdata/arocchetti/' + DATE + '/'+ material + '/' + isotope + '/' + 'output_'+ material + '_' + isotope +  '_' + num_file+'_Sort' + '.root'
        data = get_xyz(rootfile)
        flange1_in, flange2_in, flange3_in, shell_in, elongation_in = divide_innercryo(data)
        flange_in = pd.concat([flange1_in, flange2_in, flange3_in])
        #calculate_rate(flange_in, "SS_InnerCryostatFlanges", N, mass_fv)
        #calculate_rate(shell_in, "SS_InnerCryostatShell", N, mass_fv)
        #calculate_rate(elongation_in, "SS_InnerCryostat_bottomDome", N, mass_fv)
        

        
    elif material == 'SS_BellSideWall':
        rootfile = '/userdata/arocchetti/' + DATE + '/'+ material + '/' + isotope + '/' + 'output_'+ material + '_' + isotope +  '_' + num_file+'_Sort' + '.root'
        data = get_xyz(rootfile)
        bottomLip, bellWall = divide_bellWall(data)
        #calculate_rate(bottomLip, "SS_BellSideWallBottomLip", N, mass_fv)
        #calculate_rate(bellWall, "SS_BellSideWall", N, mass_fv)
        
    else: 
        rootfile = '/userdata/arocchetti/' + DATE + '/'+ material + '/' + isotope + '/' + 'output_'+ material + '_' + isotope +  '_' + num_file+'_Sort' + '.root'
        data = get_xyz(rootfile)
        calculate_rate(data, material, N, mass_fv)



IndexError: index -1 is out of bounds for axis 0 with size 0

In [ ]:
#data_test

In [ ]:
#data_test.ns